# K26 - Heated Wall

Interface at 90°.  
Equal fluid densities => simplified setting  
Also no Heat capacity => infinitely fast heat conduction  

In [ ]:
#r "..\..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
static var myDb = myBatch.CreateOrOpenCompatibleDatabase("XNSFE_HeatedWall");

In [ ]:
BoSSSshell.WorkflowMgm.Init($"HeatedWall_Simple");

## Postprocessing

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.Sessions;

In [ ]:
var dataCL = sessions.ReadLogDataForMovingContactLine()[1];
var dataEvap = sessions.ReadLogDataForXNSE("Massflux");
dataCL.Pick(4).dataGroups.ForEach(s => s.Values = s.Values.Select(v => 180 - v ).ToArray());

Plot2Ddata evapFluxes = new Plot2Ddata();
evapFluxes.dataGroups = new Plot2Ddata.XYvalues[dataEvap.Pick(3).dataGroups.Length]; // same as total evaporative heatflux * dt, as rho = hvap = 1
Plot2Ddata bndFluxes = new Plot2Ddata();
bndFluxes.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(1).dataGroups.Length]; 
Plot2Ddata.XYvalues[] Vals = new Plot2Ddata.XYvalues[dataCL.Pick(4).dataGroups.Length];
Plot2Ddata clVeloY = new Plot2Ddata();
clVeloY.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(3).dataGroups.Length]; 
int dashType = 0;
for(int i = 0; i<sessions.Count;i++){
    sessions[i].KeysAndQueries.TryGetValue("id:PressureRatio", out object color);
    color = color == null ? 7 : Convert.ToInt32(color);
    dataCL.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
    dataEvap.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
    PlotFormat PF = new PlotFormat(lineColor: (LineColors)color, dashType: ((DashTypes)(++dashType)));
    dataCL.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
    dataEvap.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
    
    double alpha = 180/Math.PI * 0.5*Math.Asin(2*Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"]));
    Vals[i] = new Plot2Ddata.XYvalues("q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]), new double[] {0.0, 15}, new double[] {alpha, alpha});
    Vals[i].Format = new PlotFormat("b-");

    double dt = Convert.ToDouble(sessions[i].KeysAndQueries["dtFixed"]);
    PF.LineWidth = 2.0;
    PF.LineColor = (LineColors)1;
    PF.DashType = (DashTypes)1;
    evapFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
    evapFluxes.dataGroups[i].Values = dataEvap.Pick(3).dataGroups[i].Values.Select(x => x).ToArray();
    evapFluxes.dataGroups[i].Abscissas = dataEvap.Pick(3).dataGroups[i].Abscissas.Select(x => x).ToArray();
    evapFluxes.dataGroups[i].Name = "Q-Evap:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    evapFluxes.dataGroups[i].Format = PF.CloneAs();
    evapFluxes.dataGroups[i].Values = evapFluxes.dataGroups[i].Values.Select(v => v / dt).ToArray();

    PF.LineColor = (LineColors)3;
    PF.DashType = (DashTypes)3;
    bndFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
    bndFluxes.dataGroups[i].Values = dataCL.Pick(1).dataGroups[i].Values.Select(x => x).ToArray();
    bndFluxes.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
    bndFluxes.dataGroups[i].Name = "Q-Bnd:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    bndFluxes.dataGroups[i].Format = PF.CloneAs();
    bndFluxes.dataGroups[i].Values = bndFluxes.dataGroups[i].Values.Select(v => v * Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"])).ToArray();

    PF.LineColor = (LineColors)color;
    PF.DashType = (DashTypes)dashType;
    clVeloY.dataGroups[i] = new Plot2Ddata.XYvalues("");
    clVeloY.dataGroups[i].Values = new double[dataCL.Pick(3).dataGroups[i].Values.Length];
    for(int j = 0; j<dataCL.Pick(3).dataGroups[i].Values.Length; j++){
        clVeloY.dataGroups[i].Values[j] = dataCL.Pick(3).dataGroups[i].Values[j] - dataCL.Pick(2).dataGroups[i].Values[j] / Math.Tan(Math.PI / 180.0 * dataCL.Pick(4).dataGroups[i].Values[j]);
    } 
    clVeloY.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
    clVeloY.dataGroups[i].Name = "Q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    clVeloY.dataGroups[i].Format = PF.CloneAs();
}
dataCL.Pick(4).dataGroups = dataCL.Pick(4).dataGroups.ToList().Cat(Vals).ToArray();    
evapFluxes = evapFluxes.Merge(bndFluxes);

In [ ]:
var gp = new Gnuplot();
var data = dataCL.Pick(4);
data.Xlabel = "time [s]";
data.Ylabel = "angle [°]";
data.ToGnuplot(gp);
gp.PlotNow()

In [ ]:
var gp = new Gnuplot();
var data = evapFluxes;
data.Xlabel = "time [s]";
data.Ylabel = "Total Heat Flux [W]";
data.ToGnuplot(gp);
gp.PlotNow()

In [ ]:
var gp = new Gnuplot();
var data = dataCL.Pick(1);
data.Xlabel = "time [s]";
data.Ylabel = "Pos Y [m]";
data.ToGnuplot(gp);
gp.PlotNow()

In [ ]:
var gp = new Gnuplot();
var data = clVeloY;
data.Xlabel = "time [s]";
data.Ylabel = "V_y [m/s]";
data.ToGnuplot(gp);
gp.PlotNow()